# 04 Going bigger

## Stages

So far our analysis has been dedicated to Stedman *Triples* on seven bells. It's possible to ring Stedman on different numbers of bells and Stedman *Cinques*, on eleven bells, is more commonly rung to *peals*<sup>1</sup> each year. The structure of Stedman Cinques is very similar to that of triples: each six the front three bells cycle while the others are fixed, and calls affect the back three bells. With more bells ringing it is harder to devise ways to reach particular course ends and so computer searches become more valuable.

Let's go part-way there and extend our solution to Stedman *Caters* on 9 bells.

<sup>1</sup> A peal of Stedman Triples must contain all 5,040 possible rows, and takes around three hours to ring. A peal of Stedman Cinques cannot contain all 39,916,800 possible rows so ringers settle for any length over 5,000.

In [1]:
from stedman_searching.profiling import Timer
from stedman_searching.rows import perm_for_rounds, perm_from_row, row_from_perm, SetRow


PAIR_TRANSITIONS = [
    (~perm_from_row('246819375'), 'pp'),
    (~perm_from_row('246815397'), 'p-'),
    (~perm_from_row('246815379'), 'ps'),
    (~perm_from_row('246718395'), '-p'),
    (~perm_from_row('246715389'), '--'),
    (~perm_from_row('246719385'), 'sp'),
    (~perm_from_row('246715398'), 's-'),
]

STARTING_ROW = SetRow(
    '123456789',
    calling='',
    perm=perm_for_rounds(8),
)

def generate_previous_row_inner(current_row, pair_transition):
    pair_transition_perm, pair_transition_calling = pair_transition

    previous_calling = pair_transition_calling + current_row.calling  # n.b. order
    previous_perm = pair_transition_perm * current_row.perm

    return SetRow(
        row_from_perm(previous_perm),
        calling=previous_calling,
        perm=previous_perm,
    )

def generate_previous_rows(current_rows):
    return set([
        generate_previous_row_inner(current_row, pair_transition)
        for current_row in current_rows
        for pair_transition in PAIR_TRANSITIONS
    ])

all_rows = set()
rows_per_iteration = [set([STARTING_ROW])]

timer = Timer()

for _ in range(7):
    current_rows = rows_per_iteration[-1]
    previous_rows = generate_previous_rows(current_rows)
    previous_rows = previous_rows - all_rows  # prune duplicates
    rows_per_iteration.append(previous_rows)
    all_rows = all_rows | previous_rows

print('elapsed time:', timer.split())
print('rows visited:', len(all_rows))

[len(rows) for rows in rows_per_iteration]

elapsed time: 2.630551680998906
rows visited: 76723


[1, 7, 41, 239, 1189, 5238, 18827, 51182]

This seems to work correctly but we have a problem. Increasing the size of the search space has revealed the limits of our algorithm. We have found all rows reachable within fourteen sixes (seven six-pairs) but it has taken us several seconds to do so. A course of Stedman Caters lasts for 18 sixes, so we haven't yet reached the end. Meanwhile we have only reached 76,723 out of 362,880 possible rows: a mere 20%.

Running the search to completion produces the following breakdown of distances after nearly a minute:

$$[1, 7, 41, 239, 1189, 5238, 18827, 51182, 102266, 127909, 51479, 4479, 24]$$

A long search time isn't necessarily an issue if we can store the generated table, but a search taking over a minute is likely to be extremely long if we add two more bells and expand the search space by a factor of 110. Moreover significantly we run into another problem:

```
MemoryError

During handling of the above exception, another exception occurred:

MemoryError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "main.py", line 47, in <module>
    previous_rows = generate_previous_rows(current_rows)
  File "main.py", line 36, in generate_previous_rows
    for current_row in current_rows
MemoryError
```

## Moving beyond sets

We're already pushing the conceptual boundaries of sets by using the `SetRow` class to stash additional data alongside each row. We're also pushing the boundaries of available memory. Let's see how much we need for Stedman Cinques:

In [2]:
from math import factorial
import sys

example_row = SetRow(
    '1234567890E',
    calling='p' * 18,
    perm=perm_for_rounds(10),
)

sys.getsizeof(example_row) * factorial(11)

1916006400

... i.e. over 2GB. I can throw more resources at the problem (and stop running searches in a small VM!), but that won't solve the timing problem. We need to do something better. In order to identify the rows furthest from rounds let's store that distance: how far away from rounds is each row?

I've built a class wrapping a [Numpy](https://numpy.org/doc/) array:

In [3]:
from stedman_searching.distances_array import DistancesArray

# Create an array with space for 4! = 24 rows
array = DistancesArray(4)

# Add some row distances
array.add(0, 2)  # index, distance
array.add(1, 4)
array.add(2, 4)
array.add(3, 4)
array.add(4, 6)
array.add(5, 6)
array.add(6, 6)
array.add(7, 6)
array.add(8, 6)

# Produce a histogram of distances
array.get_counts()

# 15 of distance 0, 1 of distance 2, 3 of distance 4, 5 of distance 6

[15, 1, 3, 5]

In order to access an array we need to be use an index. Sympy's [`Permutation`](https://docs.sympy.org/latest/modules/combinatorics/permutations.html) offers three ways to index a row:

* `rank()` which determines the _lexicographic_ rank of a row. This is the index of a row within a sorted list of all rows. This is based on assembling a [_Lehmer code_](https://en.wikipedia.org/wiki/Lehmer_code) for the permutation.
* `rank_nonlex()` based on [this algorithm](https://webhome.cs.uvic.ca/~ruskey/Publications/RankPerm/MyrvoldRuskey.pdf) which promises linear time performance.
* `rank_trotterjohnson()` using the [Steinhaus-Johnson-Trotter algorithm](https://en.wikipedia.org/wiki/Steinhaus%E2%80%93Johnson%E2%80%93Trotter_algorithm)

Let's measure which is quickest.

In [4]:
%timeit [row.perm.rank() for row in all_rows]

90.8 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit [row.perm.rank_nonlex() for row in all_rows]

546 ms ± 4.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit [row.perm.rank_trotterjohnson() for row in all_rows]

611 ms ± 2.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
